In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC
import joblib
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import re
from wordcloud import WordCloud,STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

import os
import os.path as osp

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import shutil
import time

from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'gpu'
import copy

import torch.nn as nn
from sklearn.metrics import accuracy_score, confusion_matrix


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
TRAIN_CSV = "./data/book-full-listing-train.csv"
TEST_CSV = "./data/book-full-listing-test.csv"
df = pd.read_csv(TRAIN_CSV)
df_test = pd.read_csv(TEST_CSV)


In [19]:
df.drop(['Amazon ID (ASIN)','Filename','Image URL'],axis=1,inplace=True)
df_test.drop(['Amazon ID (ASIN)','Filename','Image URL'],axis=1,inplace=True)

In [23]:
def review_to_words(raw_review):
    review = raw_review
    review = re.sub('[^a-zA-Z]', ' ',review)
    review = review.lower()
    review = review.split()
    lemmatizer = WordNetLemmatizer()
    review = [lemmatizer.lemmatize(w) for w in review if w not in set(stopwords.words('english'))]
    res = (' '.join(review))
    if res == '' or res == ' ':
        return raw_review
    else:
        return res

In [24]:
corpus= []
for i in range(0, len(df.index)):
    corpus.append(review_to_words(df['Title'][i]))
corpus_test= []
for i in range(0, len(df_test.index)):
    corpus_test.append(review_to_words(df_test['Title'][i]))    

In [25]:
df['corpus']=corpus
df_test['corpus']=corpus_test

In [26]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6577 entries, 0 to 6576
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Amazon ID (ASIN)  6577 non-null   object
 1   Filename          6577 non-null   object
 2   Image URL         6577 non-null   object
 3   Title             6577 non-null   object
 4   Author            6194 non-null   object
 5   Category ID       6577 non-null   int64 
 6   Category          6577 non-null   object
 7   label             6577 non-null   int64 
 8   resnet34mini      6577 non-null   object
 9   corpus            6577 non-null   object
dtypes: int64(2), object(8)
memory usage: 514.0+ KB


In [27]:
df.to_csv(TRAIN_CSV, index=False)
df_test.to_csv(TEST_CSV, index=False)

In [28]:
BIG_CV_JOBLIB_PATH = './joblib/big/cv.joblib'
BIG_NB_JOBLIB_PATH = './joblib/big/nb.joblib'

In [29]:
loaded_classifier = joblib.load(BIG_NB_JOBLIB_PATH)
cv = joblib.load(BIG_CV_JOBLIB_PATH)

In [33]:
x_train = cv.transform(df['corpus']).toarray()
x_test = cv.transform(df_test['corpus']).toarray()
y_train = df['Category'].values
y_test = df_test['Category'].values

In [35]:
y_pred_train = loaded_classifier.predict_proba(x_train)
y_pred_test = loaded_classifier.predict_proba(x_test)

In [34]:
y_pred = loaded_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8020374030713091


In [43]:
df['nb_cv_20000']=y_pred_train.tolist()
df_test['nb_cv_20000']=y_pred_test.tolist()

In [48]:
from typing import Any


def get_net():
    resnet = torchvision.models.resnet34(pretrained=True)

    # Substitute the FC output layer
    resnet.fc = torch.nn.Linear(resnet.fc.in_features, 10)
    torch.nn.init.xavier_uniform_(resnet.fc.weight)
    return resnet

class ModelSoftmax(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.model(x)
        y = self.softmax(x)
        return y

class ResnetVote:
    def __init__(self, model: nn.Module):
        self.model=model.to('cuda').eval()
    
    def __call__(self, x: torch.Tensor) -> Any:
        return self.model(x.to('cuda'))

In [49]:
checkpoint = torch.load('./ckpt/resnet34mini.pth.tar')
net = get_net()
net.load_state_dict(checkpoint['state_dict'])
modelsoftmax = ModelSoftmax(net)
resnetvote = ResnetVote(modelsoftmax)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
